# Reinforcement Learning Final Project #

### Caleb Stillman ###

GitHub: https://github.com/cstillma/RLFinalProject

## Project Overview ## 

In this final project for the CSPB3202 Intro to AI course, I'll be building a Deep Q-Network agent to test in the Box2D Car Racing environment available through OpenAI Gymnasium. This particular reinforcement learning algorithm has been covered in lectures but was not one that was used in any previous homeworks, so this will serve as a hands-on test of my understanding of this algorithm. I will also be including a simple random agent so I have a performance baseline to compare my DQN model to in the Car Racing environment. Time permitting, I will test the DQN agent on other OpenAI Gymnasium environments like the Lunar Landing environment to see the algorithm's performance in different settings. 

## Approach ##

### Environment ###

The environment I'm using for this project involves a Jupyter notebook for testing and observation (this notebook), with my agents called from separate python files. All of this is being utilized within the VSCode IDE. I have imported additional libraries for use in creating and testing my agents, notably numpy, gymnasium, swig, gymnasium[box2d], and tensorflow. Numpy is primarily for reshaping the state space into a readable shape for the model. Gymnasium, swig, and gymnasium[box2d] were all required for running the Car Racing environment. Tensorflow (and subsequently Keras) are used for constructing the DQN agent's model. 

### Choice of Model ###

My approach to choosing the DQN model was perhaps unconventional - rather than identifying a specific model I wanted to use first, and a suitable environment second; I spent time browsing the environments available in Gymnasium to find one that looked interesting first and then reviewed the options for models that would be best suited for learning that environment second. One of the reasons the Car Racing environment stood out to me, in addition to its complexity (it pretty much can't be solved by a random agent), is that one has the option to use either a continuous or a discrete action space with this environment, which didn't limit my potential model options too much. That said, the default action space for this environment is continuous, and this made me curious whether that choice was made because the discrete action space would be more difficult for a model to learn. Ultimately, this helped inform my choice of Deep Q-Network for my agent, because DQNs are well-suited to discrete action spaces. Furthermore, I liked the DQN as an option because it felt like a natural extension of algorithms I've used previously in this class (the approximate Q-learning agent from the Pacman homework and the convolutional neural network I used for the Kaggle competition homework). The DQN shares aspects of both, but is distinct from either as well. 

### Methods and Purpose of Testing ###

To test the DQN agent's efficacy on the Car Racing environment, I first established a baseline to which I could compare the DQN agent's eventual performance. To that end, I wrote the simple random agent present in the random_agent.py file. I chose a random agent instead of a heuristic agent to serve as a baseline because it gives me an idea of what it would look like if my DQN agent is just randomly guessing instead of learning the environment. While the heuristic agent would have provided a more challenging baseline by implementing more human-like strategies for solving the Car Racing environment, I felt that could be done later if time permitted and that a random agent provides a more "true" performance baseline. You will see that I have imported the random agent below and tested it on a limited number of episodes just to see roughly how effective it is at completing the game (the answer is not very). I then imported the DQN Agent and tested it on a small number of episodes as well. Initially, the number of episodes chosen (10), was simply to establish that the algorithm worked, without much expectation that I would see very good performance during the run, and that I would ultimately train and test the model across 1000 episodes. However, this test showed me that the agent takes an incredible amount of time and computational resources to operate (54 minutes per episode on average at my last run), and that I didn't have 1000 hours worth of time to test the agent with by the deadline of this project. As a result, I settled on only ever testing my agent on 10 episodes once I confirmed that it worked on the initial run, sacrificing long-term score tests for the opportunity to test changes in hyperparameters on the smaller batch size. Another factor behind this decision was that even that limited number of episodes demonstrated the agent's capacity compared to the random agent's baseline, which was "enough" for the purposes of this project. With unlimited time, I would have liked to see how many episodes it would have taken for my agent to reach a positive score in the game (the professor suggested it would take 800-900 episodes), but you can see for yourself below that the agent vastly outperforms the random agent's baseline after just 10 episodes.

### Troubleshooting ###

To be honest, my local Python environments were the part of this project I spent the most time troubleshooting, namely due to the fact that I had like five different versions installed and different dependencies on each one. I had difficulty finding one that I could get working with minimal effort, to the point that I decided to uninstall them all and install a fresh version. I had similar difficulties with the Kaggle competition homework but worked around them for that assignment, but I decided to take a more permanent approach to the issue this time around. I initially went with Python 3.13 because it isn't the latest release but should be more stable and supported for that reason. However, I still ran into problems here because it turns out neither TensorFlow nor PyTorch supports Python 3.13 yet; so I had to go back and install Python 3.11.9 for use with this project so that I could use TensorFlow. This resolved all of my broken/outdated dependencies and versions; it was quick work to install the exact libraries I needed and everything worked fairly smoothly after that point. 

The other primary issue I had to troubleshoot was getting the state space of the Car Racing environment into a shape that was readable by my choice of model. This took several rounds of reworking the code until the DQN Agent would run without returning a ValueError, but I got it after an hour or two. 

### Setting up the environment ###

In [1]:
import gymnasium as gym
import numpy as np
import random_agent
from dqn_agent import DQN_Agent


### Setting a Baseline ###

I'll be using a simple random agent to set a baseline for the performance of the DQN agent.

In [2]:
# Call the main function for the random agent
random_agent.RA_main()

Episode 1 finished with a score of reward: -582.9149253731705
Episode 2 finished with a score of reward: -1051.8576642337243
Episode 3 finished with a score of reward: -1150.3259259260644
Episode 4 finished with a score of reward: -1090.3798165139176
Episode 5 finished with a score of reward: -1134.9443037976214


The testing the random agent for 5 episodes shows that it performs very poorly at this game. Not a hard benchmark to beat at least. 

### Testing the DQN ###

Now I want to test the DQN to make sure it even runs. I'll be testing it on a small number of episodes at first - not enough to really see any kind of performance increase at this stage, just making sure it works before training it for 1000 episodes since that will take a while to run. 

In [2]:
# Initialize the Car Racing environment
env = gym.make("CarRacing-v3", continuous = False)

# Define the state and action sizes
state_size = env.observation_space.shape[0] # the shape of the state space
action_size = env.action_space.n # number of discrete actions

# Initialize the DQN agent
agent = DQN_Agent(state_size, action_size)

Training Loop

In [3]:
num_episodes = 10 # number of episodes to train/test

for episode in range(num_episodes):
    state, info = env.reset() # reset the environment at the start of an episode
    total_reward = 0 # track the cumulative reward for the episode
    done = False
    while not done:
        # DQN agent selects an action
        action = agent.act(state)
        # Apply the action to the environment
        next_state, reward, done, truncated, info = env.step(action)
        # Store the experience in the replay buffer
        agent.remember(state, action, reward, next_state, done)
        # Train the agent using the replay buffer
        agent.replay()
        # Update the cumulative reward
        total_reward += reward
        # Update the state
        state = next_state
    
    print(f"Episode {episode + 1}/{num_episodes} finished with total reward: {total_reward}")
env.close() # close the environment after all episodes are finished

Episode 1/10 finished with total reward: -108.88452012383893
Episode 2/10 finished with total reward: -151.1562091503273
Episode 3/10 finished with total reward: -85.96357615894098
Episode 4/10 finished with total reward: -102.74754098360711
Episode 5/10 finished with total reward: -103.11578947368474
Episode 6/10 finished with total reward: -69.21818181818227
Episode 7/10 finished with total reward: -147.264705882354
Episode 8/10 finished with total reward: -149.30546075085405
Episode 9/10 finished with total reward: -109.89636963696384
Episode 10/10 finished with total reward: -89.02876712328788


Surprisingly, 10 episodes was actually enough to establish the DQN agent's superior performance over a random agent. Obviously a great deal more episodes would be ideal for confirming that the agent can eventually pass the game with a positive score, and I'm confident it would reach that point, but you can see by the runtime of the training loop that hundreds of episodes is not feasible given a short timeframe to complete the project. At this point, I believe it necessary to continue any further testing on this small batch of episodes, since I'd like to see the effects of hyperparameter tuning - who knows, maybe I can get the runtime down low enough that 1000 episodes becomes feasible to run in one day. 

## Results ##

### Latest Result ###

The latest result you see above shows the best performance by the model yet, with scores for the 10 episodes ranging between -69.2 and -151.2. However, runtime has suffered after the last change (over 9 hours to run 10 episodes after increasing the number of neurons in the model's layers). 

### Iterative Improvements ###

Initial Hyperparameter Values:

Learning Rate: 0.001
Discount Factor: 0.95
Exploration Rate: 1.0
Minimum Exploration Rate: 0.01
Epsilon Decay: 0.995
Replay Buffer Size: 2000
Minibatch Size: 128
Number of Neurons per Layer: 12
Activation Function: ReLU

The iterative improvements I was able to make in the time I had available for this project were twofold: the first improvement was to decrease the runtime of the model, and the second was to improve the performance of the model. Of course, both changes had inverse effects on the other to varying degrees. My first run of the training loop with the above settings took roughly 16.5 hours to complete and resulted in scores ranging from -225.7 to -306.3 for the 10 episodes. This was an atrocious runtime, and while the performance was better than the baseline set by the random agent, it was not by much in some episodes. 

This is why the first change I prioritized was to improve the runtime. To accomplish this, I considered several of the hyperparameters for adjustment, but I didn't want the performance to suffer too much. Ultimately, I chose to drastically reduce the minibatch size from 128 experiences to 32, as a larger minibatch for training the model can provide more stable training to the agent, but at the expense of computational cost and speed. This change did dramatically reduce the runtime of the training loop; runtime was 7 hours and 13 minutes but I did see that the performance suffered somewhat, with scores ranging from -268.8 to -413.9 across the 10 episodes. I was left wondering if reducing the replay buffer size to 1000 experiences in memory would have been the better choice for reducing runtime without sacrificing as much performance-wise. I had considered changing both for the sake of the time I had left in the project, but decided that I couldn't have done more than speculate on which was more responsible for the resulting scores and runtime, so I chose to only pick one for the sake of cleaner testing. 

This leads to the second improvement. While I had improved the runtime considerably, I wanted to see better performance from the model. I was sure that I wouldn't see positive scores in my 10 test episodes no matter what changes I made because the professor flat out said that this model type needs hundreds of episodes to learn enough to start seeing positive game scores. That said, while I knew I was out of time to see that for myself, I wanted to feel confident that my model would reach that point given enough episodes if I ever did run that many episodes, so I chose to attempt to improve performance for my second hyperparameter tuning. To accomplish this, I immediately knew which parameter I wanted to tune - I had chosen a low number of neurons per layer initially as an attempt to keep runtime lower because I had no idea how long this agent would run and was counting on the initial epsilon, gamma, and minibatch values to drive the model's first performance, but I knew that this was the first thing I would want to increase if I wanted to improve performance. Therefore, I increased the number of neurons in my model's hidden layers from 12 to 24 each. I wanted the model to be better at both identifying basic patterns in the state space (the first hidden layer), and better at combining these patterns for more performant decision-making by increasing the neurons in the second hidden layer. I think that the results speak for themselves with scores between -69.2 and -151.2. This did come at a cost, increasing the runtime back up to just over 9 hours, or ~54 minutes per episode on average. 

This is unfortunately where I have to stop testing due to time constraints. 



## Video Demo ##

## Conclusion ##

In conclusion, my third iteration of the training loop proves to me beyond a doubt that the DQN model is capable of truly learning the environment of the Car Racing game, vastly outperforming the random agent baseline, and I'm extremely hopeful that this iteration would achieve positive scores given the time to run 1000 episodes. Given unlimited time, the hyperparameters that I would adjust next would be to reduce the number of experiences stored in the memory buffer - I speculate that if I reduced that from storing 2000 experiences to 1000, that it would reduce memory usage and runtime further without compromising the performance I've achieved in this latest round. I chose 2000 experiences back when I thought I would have time not only to test on the Car Racing game, but to move on to testing the DQN agent to the Lunar Landing game as well, and I wanted the model to store enough experiences to generalize well across different environments instead of just memorizing the Car Racing environment's patterns. However, I was obviously underestimating the runtimes of these more complex models on the more complex environments available in the Gymnasium based on my experience with the approximate Q-learning agent on the Pacman environment. I was especially shocked by the runtimes I was experiencing here compared to the CNN runtimes on the histopathic cancer detection assignment because the dataset was so large there - I was sure that the gigabytes worth of data that model needed to crunch would mean the CNN's runtimes would be far longer than anything I would encounter in this project. 